In [525]:
import json
import re
from tqdm import tqdm
import os
import pandas as pd
import subprocess
from pathlib import Path
import shutil

In [526]:
def get_commit_timestamp(commit_hash="HEAD"):
    """Получить Unix timestamp коммита"""

    result = subprocess.run(
        ["git", "show", "-s", "--format=%ct", commit_hash],
        capture_output=True,
        text=True,
        check=True
    )
    return int(result.stdout.strip())

def checkout(commit_hash: str):
    subprocess.run(
        ["git", "checkout", commit_hash]
    )
    


In [527]:
data = pd.read_parquet('/Users/nikita/Desktop/swe_agent_deploy/SWE-bench_Verified/data/test-00000-of-00001.parquet')
data.FAIL_TO_PASS = data.FAIL_TO_PASS.apply(eval)
data.PASS_TO_PASS = data.PASS_TO_PASS.apply(eval)

with open('/Users/nikita/Desktop/swe_agent_deploy/SWE-Bench_Scoring/Verified/gpt-5-mini/preds.json', 'r') as fp:
    preds = json.load(fp)
preds = {item['instance_id']:item for item in preds.values() if item['model_patch'].startswith('diff --git')}

reports = []
for inst_id in data.instance_id:
    try:
        with open(f'/Users/nikita/Desktop/swe_agent_deploy/SWE-bench/logs/run_evaluation/'
                f'test_evaluation/openrouter__gpt-5-mini/{inst_id}/report.json', 'r') as fp:
            report = json.load(fp)
        key, value = next(iter(report.items()))
        value['instance_id'] = key
        
        with open(f'/Users/nikita/Desktop/swe_agent_deploy/SWE-bench/logs/run_evaluation/'
            f'test_evaluation/openrouter__gpt-5-mini/{inst_id}/test_output.txt', 'r') as fp:
            test_opuput = fp.read()
        result = re.search(">>>>> Start Test Output(.*?)>>>>> End Test Output", test_opuput, re.DOTALL)
        value['tests_logs'] = result.group().replace(">>>>> Start Test Output'\n", '').replace("\n+ : '>>>>> End Test Output", '')

        reports.append(value)
    except:
        print(inst_id)
    
report_df = pd.DataFrame(reports)
report_df['FAIL_TO_PASS_failed_tests'] = report_df.tests_status.apply(lambda x: x['FAIL_TO_PASS']['failure'])
report_df['PASS_TO_PASS_failed_tests'] = report_df.tests_status.apply(lambda x: x['PASS_TO_PASS']['failure'])
report_df = report_df.drop(columns=['tests_status'])


psf__requests-1142
sphinx-doc__sphinx-7590
sympy__sympy-13480


In [528]:
df = data.merge(pd.DataFrame(preds.values())).merge(report_df)

In [529]:
os.chdir('/Users/nikita/Desktop/swe_agent_deploy/repos')
for repo in df.repo.unique():
    repo_name = repo.split('/')[1]
    if os.path.exists(repo_name):
        continue
    else:
        subprocess.run(['git', 'clone', f'https://github.com/{repo}.git'])

In [530]:
os.chdir('/Users/nikita/Desktop/swe_agent_deploy/repo_snapshots')
for repo in df.repo.unique():
    repo_name = repo.split('/')[1]
    if os.path.exists(repo_name):
        continue
    else:
        os.mkdir(repo_name)

In [540]:
os.chdir('/Users/nikita/Desktop/swe_agent_deploy/bench_repos')
for repo in df.repo.unique():
    repo_name = repo.split('/')[1]
    if os.path.exists(repo_name):
        continue
    else:
        os.mkdir(repo_name)

In [532]:
timestamps = []
for i in tqdm(range(len(df))):
    repo_name = df.iloc[i]['repo'].split('/')[1]
    os.chdir(f'/Users/nikita/Desktop/swe_agent_deploy/repos/{repo_name}')
    timestamps.append(get_commit_timestamp(df.iloc[i]['base_commit']))

df['base_timestamp'] = timestamps

100%|██████████| 497/497 [00:13<00:00, 37.39it/s]


In [533]:
for i in tqdm(range(len(df))):
    repo_name = df.iloc[i]['repo'].split('/')[1]
    target_folder = f'/Users/nikita/Desktop/swe_agent_deploy/repo_snapshots/{repo_name}/{df.iloc[i]['base_commit']}'
    if os.path.exists(target_folder):
        continue
    else:
        os.chdir(f'/Users/nikita/Desktop/swe_agent_deploy/repos/{repo_name}')
        checkout(df.iloc[i]['base_commit'])
        for py_file in Path('.').rglob('*.py'):
            if py_file.is_file():
                relative_path = py_file.relative_to('.')
                dst_path = Path(target_folder) / relative_path
                dst_path.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(py_file, dst_path)

100%|██████████| 497/497 [00:00<00:00, 18029.33it/s]


In [541]:
class GitCommiter:
    def __init__(self, repo_path, snapshots_dir):
        self.repo_path = Path(repo_path)
        self.snapshots_dir = Path(snapshots_dir)
        self.commit_mapping = {}
        
    def is_git_repo(self):
        """Проверяет, является ли папка git репозиторием"""
        git_dir = self.repo_path / ".git"
        return git_dir.exists() and git_dir.is_dir()
    
    def initialize_repo(self):
        """Инициализирует git репозиторий если его нет"""
        if not self.is_git_repo():
            print(f"Инициализация репозитория в {self.repo_path}")
            result = subprocess.run(["git", "init"], 
                                  cwd=self.repo_path, 
                                  capture_output=True, 
                                  text=True)                  
            if result.returncode != 0:
                raise Exception(f"Ошибка инициализации репозитория: {result.stderr}")

            # Принудительно создаем ветку main
            result = subprocess.run(["git", "branch", "-M", "main"], 
                                  cwd=self.repo_path, 
                                  capture_output=True, 
                                  text=True)
            if result.returncode != 0:
                print(f"Предупреждение: не удалось переименовать ветку в main: {result.stderr}")

            print("Репо инициализирован успешно")
        else:
            print("Репо уже инициализирован")
    
    def copy_snapshot(self, snapshot_name):
        """Копирует содержимое снепшота в репозиторий"""
        snapshot_path = self.snapshots_dir / snapshot_name
        if not snapshot_path.exists():
            raise Exception(f"Снепшот {snapshot_name} не существует")
        
        # Очищаем репозиторий (кроме .git папки)
        for item in self.repo_path.iterdir():
            if item.name != ".git":
                if item.is_file():
                    item.unlink()
                else:
                    shutil.rmtree(item)
        
        # Копируем содержимое снепшота
        for item in snapshot_path.iterdir():
            dest = self.repo_path / item.name
            if item.is_file():
                shutil.copy2(item, dest)
            else:
                shutil.copytree(item, dest)

    def apply_test_patch(self, patch: str):
        
        result = subprocess.run(
            f"git apply << 'EOF'\n{patch}\nEOF",
            shell=True,
            check=True
        )

        if result.returncode != 0:
            print(f"Предупреждение: не удалось применить test patch: {result.stderr}")
            return 
        else:
            result = subprocess.run(['git', 'add', '**/*.py'])
            if result.returncode != 0:
                print(f"Предупреждение: не удалось применить test patch: {result.stderr}")

    def has_changes(self):
        """Проверяет, есть ли изменения для коммита"""
        result = subprocess.run(["git", "status", "--porcelain"], 
                              cwd=self.repo_path, 
                              capture_output=True, 
                              text=True)
        return bool(result.stdout.strip())
    
    def create_commit(self, commit_message):
        """Создает коммит с заданным сообщением"""
        # Добавляем все файлы
        add_result = subprocess.run(["git", "add", "."], 
                                  cwd=self.repo_path, 
                                  capture_output=True, 
                                  text=True)
        if add_result.returncode != 0:
            raise Exception(f"Ошибка добавления файлов: {add_result.stderr}")

        if self.has_changes():
            # Создаем коммит
            commit_result = subprocess.run(["git", "commit", "-m", commit_message], 
                                        cwd=self.repo_path, 
                                        capture_output=True, 
                                        text=True)
            if commit_result.returncode != 0:
                raise Exception(f"Ошибка создания коммита: {commit_result.stderr}")
        else:
            print("Нет изменений для коммита, используем последний хэш")
        
        # Получаем хэш коммита
        hash_result = subprocess.run(["git", "rev-parse", "HEAD"], 
                                   cwd=self.repo_path, 
                                   capture_output=True, 
                                   text=True)
        if hash_result.returncode != 0:
            raise Exception(f"Ошибка получения хэша коммита: {hash_result.stderr}")
        
        return hash_result.stdout.strip()


    def process_snapshots(self,
                          snapshots: list[str],
                          patches: list[str],
                          timestamps: list[int]):
        """Обрабатывает все снепшоты и создает коммиты"""

        self.initialize_repo()
        commits_time_sorted = [item[1] for item in sorted(zip(timestamps, zip(snapshots, patches)))]
        
        for i, (snapshot, patch) in enumerate(commits_time_sorted, 1):
            print(f"Обработка снепшота: {snapshot}")
            
            self.copy_snapshot(snapshot)
            self.apply_test_patch(patch)
            commit_hash = self.create_commit(f"commit #{i}")
            self.commit_mapping[snapshot] = commit_hash
            
            print(f"Создан коммит #{i}: {commit_hash} для снепшота '{snapshot}'")
        print(f"Обработано {len(snapshots)} снепшотов")

In [542]:
commit_mapper = {}
for repo in ['astropy/astropy', 'django/django', 'matplotlib/matplotlib',
             'mwaskom/seaborn', 'pallets/flask', 'psf/requests',
             'pydata/xarray', 'pylint-dev/pylint', 'pytest-dev/pytest',
             'scikit-learn/scikit-learn', 'sphinx-doc/sphinx', 'sympy/sympy']:

    repo_name = repo.split('/')[1]
    commiter = GitCommiter(
        repo_path = f'/Users/nikita/Desktop/swe_agent_deploy/bench_repos/{repo_name}/',
        snapshots_dir = f'/Users/nikita/Desktop/swe_agent_deploy/repo_snapshots/{repo_name}/'
    )

    commiter.process_snapshots(
        snapshots = df.loc[df.repo == repo, 'base_commit'].tolist(),
        patches = df.loc[df.repo == repo, 'test_patch'].tolist(),
        timestamps = df.loc[df.repo == repo, 'base_timestamp'].tolist(),
    )

    commit_mapper.update(commiter.commit_mapping)
    
df = df.merge(pd.DataFrame(data = commit_mapper.items(), columns = ['base_commit', 'bench_commit']))

Инициализация репозитория в /Users/nikita/Desktop/swe_agent_deploy/bench_repos/astropy
Репо инициализирован успешно
Обработка снепшота: 26d147868f8a891a6009a25cd6a8576d2e1bd747


error: astropy/utils/tests/test_misc.py: No such file or directory


CalledProcessError: Command 'git apply << 'EOF'
diff --git a/astropy/utils/tests/test_misc.py b/astropy/utils/tests/test_misc.py
--- a/astropy/utils/tests/test_misc.py
+++ b/astropy/utils/tests/test_misc.py
@@ -80,14 +80,26 @@ def __call__(self, *args):
             "FOO"
             pass
 
+        @property
+        def bar(self):
+            "BAR"
+            pass
+
     class Subclass(Base):
         def __call__(self, *args):
             pass
 
+        @property
+        def bar(self):
+            return 42
+
     if Base.__call__.__doc__ is not None:
         # TODO: Maybe if __doc__ is None this test should be skipped instead?
         assert Subclass.__call__.__doc__ == "FOO"
 
+    if Base.bar.__doc__ is not None:
+        assert Subclass.bar.__doc__ == "BAR"
+
 
 def test_set_locale():
     # First, test if the required locales are available

EOF' returned non-zero exit status 1.

In [519]:
print(df.loc[df.test_patch.apply(len) == 367, 'test_patch'].item())

diff --git a/testing/test_mark_expression.py b/testing/test_mark_expression.py
--- a/testing/test_mark_expression.py
+++ b/testing/test_mark_expression.py
@@ -130,6 +130,7 @@ def test_syntax_errors(expr: str, column: int, message: str) -> None:
         "123.232",
         "True",
         "False",
+        "None",
         "if",
         "else",
         "while",



In [294]:
'aa::aaa::2'.split('::', 1)

['aa', 'aaa::2']

In [492]:
import ast
import importlib.util
import sys
import re

def parse_test_function(repo_path, test_identifier):
    """Парсит конкретную тестовую функцию используя AST"""
    
    if '::' in test_identifier:

        if '[' in test_identifier:
            test_identifier = test_identifier.split('[')[0]

        path, function_name = test_identifier.split('::')[0], test_identifier.split('::')[-1]
        files = [path]

    elif re.search(r'^(\w+) \(([\w_\.]+)\.(\w+)\)$', test_identifier):
        
        function_name, file_path = test_identifier.split(' ')[0], test_identifier.split(' ')[-1]
        files = ['/'.join(['tests'] + file_path.replace('(', '').replace(')', '').split('.')[:-1]) + '.py']

    elif re.search(r'^test_\w+$', test_identifier):
        function_name = test_identifier
        search_pattern = rf"def {test_identifier}\s*\("
        result = subprocess.run(
            ['grep', '-r', '-l', '-E', search_pattern, '.'],
            capture_output=True,
            text=True,
            check=True
        )
        
        files = []
        for line in result.stdout.strip().split('\n'):
            if line and line.endswith('.py'):
                if line.startswith('./'):
                    files.append(line[2:])
                else:
                    files.append(line)
    else:
        print(repo_path, 'Error in reading:', test_identifier)
        return f"Invalid test identifier: {test_identifier}"
    
    
    full_paths = [os.path.join(repo_path, file_path) for file_path in files]
    functions = []

    for fpath in full_paths:
        if not os.path.exists(fpath):
            print(repo_path, 'File not found:', fpath)
            # return f"File not found: {full_path}"

        try:
            flag = False
            with open(fpath, 'r', encoding='utf-8') as f:
                content = f.read()
            
            tree = ast.parse(content)
            
            for node in ast.walk(tree):
                if isinstance(node, ast.FunctionDef) and node.name == function_name:
                    # Получаем код функции из исходного файла
                    start_line = node.lineno - 1
                    # Если есть декораторы, начинаем с первого декоратора
                    if node.decorator_list:
                        first_decorator = node.decorator_list[0]
                        start_line = first_decorator.lineno - 1

                    end_line = node.end_lineno
                    lines = content.split('\n')[start_line:end_line]
                    functions.append('\n'.join(lines))
                    flag = True
            
            if not flag:
                print(f"Function {function_name} not found in {fpath}")
            
        except Exception as e:
            print(f"Error parsing file: {e}")
    return functions

In [493]:
df['PASS_TO_PASS_failed_tests_impl'] = [dict() for _ in range(len(df))]
for row in df.iterrows():
    test_ids = row[1]['PASS_TO_PASS_failed_tests']
    if len(test_ids) == 0:
        continue
    else:
        repo_name = row[1]['repo'].split('/')[1]
        repo_path = f'/Users/nikita/Desktop/swe_agent_deploy/bench_repos/{repo_name}'
        os.chdir(repo_path)
        bench_commit = row[1]['bench_commit']
        checkout(bench_commit)

        df.loc[df.instance_id == row[1]['instance_id'], 'PASS_TO_PASS_failed_tests_impl'] = [{test_id: parse_test_function(repo_path, test_id) for test_id in test_ids}]

Previous HEAD position was 0089c71 commit #22
HEAD is now at 6f2109d commit #19
Previous HEAD position was 6f2109d commit #19
HEAD is now at 602cb8c commit #21
Previous HEAD position was 602cb8c commit #21
HEAD is now at 0089c71 commit #22
Previous HEAD position was a3d9cf95 commit #226
HEAD is now at 0a9c6b14 commit #14
Previous HEAD position was 0a9c6b14 commit #14
HEAD is now at 5769b4b8 commit #22


/Users/nikita/Desktop/swe_agent_deploy/bench_repos/django Error in reading: Regression for #13095: Using base forms with widgets defined in Meta should not raise errors.
/Users/nikita/Desktop/swe_agent_deploy/bench_repos/django Error in reading: A custom formfield_callback is used if provided
/Users/nikita/Desktop/swe_agent_deploy/bench_repos/django Error in reading: Regression for #15315: modelform_factory should accept widgets
/Users/nikita/Desktop/swe_agent_deploy/bench_repos/django Error in reading: Regression for #19733
/Users/nikita/Desktop/swe_agent_deploy/bench_repos/django Error in reading: Data for a ManyToManyField is a list rather than a lazy QuerySet.
/Users/nikita/Desktop/swe_agent_deploy/bench_repos/django Error in reading: UserCreationForm password validation uses all of the form's data.
/Users/nikita/Desktop/swe_agent_deploy/bench_repos/django Error in reading: An invalid login doesn't leak the inactive status of a user.
/Users/nikita/Desktop/swe_agent_deploy/bench_rep

Previous HEAD position was 5769b4b8 commit #22
HEAD is now at e90d895d commit #27
Previous HEAD position was e90d895d commit #27
HEAD is now at a13a7d22 commit #30
Previous HEAD position was a13a7d22 commit #30
HEAD is now at 7a3ea337 commit #35
Previous HEAD position was 7a3ea337 commit #35
HEAD is now at d37dcd89 commit #36
Previous HEAD position was d37dcd89 commit #36
HEAD is now at 0d213237 commit #40
Previous HEAD position was 0d213237 commit #40
HEAD is now at f95c5e7a commit #44
Previous HEAD position was f95c5e7a commit #44
HEAD is now at 65060e5c commit #65


/Users/nikita/Desktop/swe_agent_deploy/bench_repos/django Error in reading: m2m fields help text as it applies to admin app (#9321).


Previous HEAD position was 65060e5c commit #65
HEAD is now at 16de26c8 commit #70
Previous HEAD position was 16de26c8 commit #70
HEAD is now at e4adbc3a commit #76


...

Previous HEAD position was a09af1a commit #25
HEAD is now at 641c5b6 commit #26
Previous HEAD position was 641c5b6 commit #26
HEAD is now at f42574b commit #28
Previous HEAD position was f42574b commit #28
HEAD is now at 0b8cb7d commit #29
Previous HEAD position was 0b8cb7d commit #29
HEAD is now at 35d728e commit #30
Previous HEAD position was 35d728e commit #30
HEAD is now at dfa1083 commit #31
Previous HEAD position was dfa1083 commit #31


Function test_viewcode_epub_enabled not found in /Users/nikita/Desktop/swe_agent_deploy/bench_repos/sphinx/tests/test_ext_viewcode.py


HEAD is now at d016cc6 commit #32
Previous HEAD position was d016cc6 commit #32
HEAD is now at 93381b1 commit #33
Previous HEAD position was 93381b1 commit #33
HEAD is now at f9e0f88 commit #34
Previous HEAD position was f9e0f88 commit #34
HEAD is now at f01e85a commit #35
Previous HEAD position was f01e85a commit #35
HEAD is now at 3a0c221 commit #36
Previous HEAD position was 3a0c221 commit #36
HEAD is now at 87800ed commit #37
Previous HEAD position was 87800ed commit #37
HEAD is now at df5e6c4 commit #38
Previous HEAD position was df5e6c4 commit #38
HEAD is now at 6140a95 commit #39
Previous HEAD position was 6140a95 commit #39
HEAD is now at 2ade24e commit #40
Previous HEAD position was 2ade24e commit #40
HEAD is now at d7bed2f commit #41


Function test_stringify_mock not found in /Users/nikita/Desktop/swe_agent_deploy/bench_repos/sphinx/tests/test_util_typing.py


Previous HEAD position was d7bed2f commit #41
HEAD is now at 9a9e71b commit #42
Previous HEAD position was ad4abcb commit #69
HEAD is now at 73774d9 commit #5
Previous HEAD position was 73774d9 commit #5
HEAD is now at 6b98de2 commit #10
Previous HEAD position was 6b98de2 commit #10
HEAD is now at f10e824 commit #16
Previous HEAD position was f10e824 commit #16
HEAD is now at 70dd949 commit #18
Previous HEAD position was 70dd949 commit #18
HEAD is now at 7be1db4 commit #36
Previous HEAD position was 7be1db4 commit #36
HEAD is now at f87b0ee commit #43
Previous HEAD position was f87b0ee commit #43
HEAD is now at 36e8c49 commit #50
Previous HEAD position was 36e8c49 commit #50
HEAD is now at 7f9e0fb commit #51
Previous HEAD position was 7f9e0fb commit #51
HEAD is now at e9709b2 commit #56
Previous HEAD position was e9709b2 commit #56
HEAD is now at baa6479 commit #60
Previous HEAD position was baa6479 commit #60
HEAD is now at ad4abcb commit #69


In [501]:
# row = df.loc[df.repo == 'pytest-dev/pytest']
row = df.loc[df.bench_commit.apply(lambda x: x.startswith('2ade24e'))]
row

row = df.iloc[419]
row['PASS_TO_PASS_failed_tests']

['tests/test_util_typing.py::test_restify',
 'tests/test_util_typing.py::test_restify_type_hints_containers',
 'tests/test_util_typing.py::test_restify_type_hints_Callable',
 'tests/test_util_typing.py::test_restify_type_hints_Union',
 'tests/test_util_typing.py::test_restify_type_hints_typevars',
 'tests/test_util_typing.py::test_restify_type_hints_custom_class',
 'tests/test_util_typing.py::test_restify_type_hints_alias',
 'tests/test_util_typing.py::test_restify_type_ForwardRef',
 'tests/test_util_typing.py::test_restify_type_Literal',
 'tests/test_util_typing.py::test_restify_pep_585',
 'tests/test_util_typing.py::test_restify_broken_type_hints',
 'tests/test_util_typing.py::test_stringify',
 'tests/test_util_typing.py::test_stringify_type_hints_containers',
 'tests/test_util_typing.py::test_stringify_type_hints_pep_585',
 'tests/test_util_typing.py::test_stringify_Annotated',
 'tests/test_util_typing.py::test_stringify_type_hints_string',
 'tests/test_util_typing.py::test_stringif

repo_path = f'/Users/nikita/Desktop/swe_agent_deploy/bench_repos/{repo_name}'
os.chdir(repo_path)
bench_commit = row[1]['bench_commit']
checkout(bench_commit)

In [502]:
repo_name = row['repo'].split('/')[1]
repo_path = f'/Users/nikita/Desktop/swe_agent_deploy/bench_repos/{repo_name}'
os.chdir(repo_path)
bench_commit = row['bench_commit']
checkout(bench_commit)

Previous HEAD position was 9a9e71b commit #42
HEAD is now at 2ade24e commit #40


In [510]:
row

repo                                                              sphinx-doc/sphinx
instance_id                                                 sphinx-doc__sphinx-9658
base_commit                                232dbe41c5250eb7d559d40438c4743483e95f15
patch                             diff --git a/sphinx/ext/autodoc/mock.py b/sphi...
test_patch                        diff --git a/tests/test_util_typing.py b/tests...
problem_statement                 Inherited classes not correctly documented whe...
hints_text                                                                         
created_at                                                     2021-09-20T17:04:23Z
version                                                                         4.3
FAIL_TO_PASS                         [tests/test_util_typing.py::test_restify_mock]
PASS_TO_PASS                      [tests/test_util_typing.py::test_restify, test...
environment_setup_commit                   6c6cc8a6f50b18331cb818160d168d7bb

In [509]:
subprocess.run(
    f"git apply << 'EOF'\n{row['test_patch']}\nEOF",
    shell=True,
    check=True
)

CompletedProcess(args='git apply << \'EOF\'\ndiff --git a/tests/test_util_typing.py b/tests/test_util_typing.py\n--- a/tests/test_util_typing.py\n+++ b/tests/test_util_typing.py\n@@ -17,6 +17,7 @@\n \n import pytest\n \n+from sphinx.ext.autodoc import mock\n from sphinx.util.typing import restify, stringify\n \n \n@@ -170,6 +171,12 @@ def test_restify_broken_type_hints():\n     assert restify(BrokenType) == \':py:class:`tests.test_util_typing.BrokenType`\'\n \n \n+def test_restify_mock():\n+    with mock([\'unknown\']):\n+        import unknown\n+        assert restify(unknown.secret.Class) == \':py:class:`unknown.secret.Class`\'\n+\n+\n def test_stringify():\n     assert stringify(int) == "int"\n     assert stringify(str) == "str"\n@@ -294,3 +301,9 @@ def test_stringify_type_union_operator():\n \n def test_stringify_broken_type_hints():\n     assert stringify(BrokenType) == \'tests.test_util_typing.BrokenType\'\n+\n+\n+def test_stringify_mock():\n+    with mock([\'unknown\']):\n+     

In [ ]:
git apply -v - <<'EOF_114329324912'

In [484]:
test_ids = row['PASS_TO_PASS_failed_tests']

In [485]:
test_ids

['tests/test_markup.py::test_inline[verify-:rfc:`2324`-<p><span',
 'tests/test_markup.py::test_inline[verify-:rfc:`2324#id1`-<p><span',
 'tests/test_markup.py::test_inline[verify_re-``code',
 'tests/test_markup.py::test_inline[verify-:menuselection:`a',
 'tests/test_markup.py::test_inline[verify-:menuselection:`&Foo',
 'tests/test_markup.py::test_inline[verify-:guilabel:`&Foo',
 'tests/test_markup.py::test_inline[verify-:guilabel:`Foo`-<p><span',
 'tests/test_markup.py::test_inline[verify-:kbd:`space`-<p><kbd',
 'tests/test_markup.py::test_inline[verify-:kbd:`Control+X`-<p><kbd',
 'tests/test_markup.py::test_inline[verify-:kbd:`M-x',
 'tests/test_markup.py::test_inline[verify_re-:option:`--with-option`-<p><code(',
 'tests/test_markup.py::test_inline[verify-"John"-<p>\\u201cJohn\\u201d</p>-\\u201cJohn\\u201d]',
 'tests/test_markup.py::test_inline[verify-``"John"``-<p><code',
 'tests/test_markup.py::test_inline[verify-:manpage:`mp(1)`-<p><em',
 'tests/test_markup.py::test_inline[verify-\

In [486]:
res = [{test_id: parse_test_function(repo_path, test_id) for test_id in test_ids}]

In [490]:
print(res[0]['tests/test_markup.py::test_inline[verify-:rfc:`2324`-<p><span'][0])

@pytest.mark.parametrize('type,rst,html_expected,latex_expected', [
    (
        # pep role
        'verify',
        ':pep:`8`',
        ('<p><span class="target" id="index-0"></span><a class="pep reference external" '
         'href="http://www.python.org/dev/peps/pep-0008"><strong>PEP 8</strong></a></p>'),
        ('\\index{Python Enhancement Proposals@\\spxentry{Python Enhancement Proposals}'
         '!PEP 8@\\spxentry{PEP 8}}\\sphinxhref{http://www.python.org/dev/peps/pep-0008}'
         '{\\sphinxstylestrong{PEP 8}}')
    ),
    (
        # pep role with anchor
        'verify',
        ':pep:`8#id1`',
        ('<p><span class="target" id="index-0"></span><a class="pep reference external" '
         'href="http://www.python.org/dev/peps/pep-0008#id1">'
         '<strong>PEP 8#id1</strong></a></p>'),
        ('\\index{Python Enhancement Proposals@\\spxentry{Python Enhancement Proposals}'
         '!PEP 8\\#id1@\\spxentry{PEP 8\\#id1}}\\sphinxhref'
         '{http://www.python.or

In [472]:
with open('testing/test_reports.py', 'r', encoding='utf-8') as f:
    content = f.read()
function_name = 'test_unserialization_failure'    
tree = ast.parse(content)
flag = False       
for node in ast.walk(tree):
    if isinstance(node, ast.FunctionDef) and node.name == function_name:
        # Получаем код функции из исходного файла
        start_line = node.lineno - 1
        # Если есть декораторы, начинаем с первого декоратора
        if node.decorator_list:
            first_decorator = node.decorator_list[0]
            start_line = first_decorator.lineno - 1

        end_line = node.end_lineno
        lines = content.split('\n')[start_line:end_line]
        functions.append('\n'.join(lines))
        flag = True
flag

NameError: name 'functions' is not defined

In [471]:
ast.Method

AttributeError: module 'ast' has no attribute 'Method'

In [469]:
tree.children

AttributeError: 'Module' object has no attribute 'children'

In [473]:
row

repo                                                              pytest-dev/pytest
instance_id                                                 pytest-dev__pytest-5787
base_commit                                955e54221008aba577ecbaefa15679f6777d3bf8
patch                             diff --git a/src/_pytest/reports.py b/src/_pyt...
test_patch                        diff --git a/testing/code/test_code.py b/testi...
problem_statement                 exception serialization should include chained...
hints_text                        currently exception serialization is best desc...
created_at                                                     2019-08-26T16:43:31Z
version                                                                         5.1
FAIL_TO_PASS                      [testing/test_reports.py::TestReportSerializat...
PASS_TO_PASS                      [testing/code/test_code.py::test_ne, testing/c...
environment_setup_commit                   c1361b48f83911aa721b21a4515a54465

In [408]:
df.loc[df.repo == 'sympy/sympy', 'PASS_TO_PASS']

423    [test_point3D, test_Point2D, test_issue_9214, ...
424    [test_no_args, test_single_arg, test_list_args...
425    [test_shape, test_matexpr, test_subs, test_Zer...
426    [test_Permutation, test_josephus, test_ranking...
427    [test_Permutation, test_josephus, test_ranking...
                             ...                        
492    [test_str_repr, test_eq, test_convert_to, test...
493                                  [test_isomorphisms]
494    [test_PolyRing___init__, test_PolyRing___hash_...
495    [test_seterr, test_mod, test_divmod, test_igcd...
496    [test_sympy_parser, test_rationalize, test_fac...
Name: PASS_TO_PASS, Length: 74, dtype: object

In [380]:
df.PASS_TO_PASS_failed_tests_impl.apply(len).max()

np.int64(548)

In [403]:
df.loc[df.PASS_TO_PASS_failed_tests.apply(len) == 548, 'PASS_TO_PASS_failed_tests_impl'].item()

{'test_missing_passwords (auth_tests.test_forms.AdminPasswordChangeFormTest)': "    def test_missing_passwords(self):\n        user = User.objects.get(username='testclient')\n        data = {'password1': '', 'password2': ''}\n        form = AdminPasswordChangeForm(user, data)\n        required_error = [Field.default_error_messages['required']]\n        self.assertEqual(form.errors['password1'], required_error)\n        self.assertEqual(form.errors['password2'], required_error)",
 'test_non_matching_passwords (auth_tests.test_forms.AdminPasswordChangeFormTest)': "    def test_non_matching_passwords(self):\n        user = User.objects.get(username='testclient')\n        data = {'password1': 'password1', 'password2': 'password2'}\n        form = AdminPasswordChangeForm(user, data)\n        self.assertEqual(form.errors['password2'], [form.error_messages['password_mismatch']])",
 'test_one_password (auth_tests.test_forms.AdminPasswordChangeFormTest)': "    def test_one_password(self):\n    

In [370]:
df.loc[df.PASS_TO_PASS_failed_tests.apply(len) == 548, 'instance_id'].item()


'django__django-11276'

In [372]:
df.loc[df.PASS_TO_PASS_failed_tests.apply(len) == 548]

,repo,instance_id,base_commit,patch,test_patch,problem_statement,hints_text,created_at,version,FAIL_TO_PASS,...,patch_is_None,patch_exists,patch_successfully_applied,resolved,tests_logs,FAIL_TO_PASS_failed_tests,PASS_TO_PASS_failed_tests,base_timestamp,bench_commit,PASS_TO_PASS_failed_tests_impl
43,django/django,django__django-11276,28d5262fa3315690395f04e3619ed554dbaf725b,diff --git a/django/utils/html.py b/django/uti...,diff --git a/tests/admin_docs/test_views.py b/...,Use Python stdlib html.escape() to in django.u...,,2019-04-24T12:35:05Z,3.0,[test_make_list02 (template_tests.filter_tests...,...,False,True,True,False,+ ./tests/runtests.py --verbosity 2 --settings...,[test_make_list02 (template_tests.filter_tests...,[test_missing_passwords (auth_tests.test_forms...,1556186165,5769b4b87ad1d08c632788db73a91e46b4374fe0,{'astropy/io/fits/tests/test_header.py::TestHe...
